# Packages and Data Importing

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from scipy import sparse,stats
import xgboost as xgb
from sklearn import preprocessing, pipeline, metrics
import time

Data Importing

In [ ]:
properties = pd.read_csv('data/properties_2016.csv')

In [ ]:
train = pd.read_csv("data/train_2016_v2.csv")

In [ ]:
# pd.read_csv(sio, dtype={"user_id": int, "username": object})

In [ ]:
properties.head()

In [ ]:
train.head()

In [ ]:
print(properties.dtypes)

In [ ]:
print(train.dtypes)

join train with properties

In [ ]:
train_df = train.merge(properties, how='left', on='parcelid')

# EDA - Exploratory Data Analysis

## Logerror Histogram

In [ ]:
plt.figure(figsize = (16, 6))
plt.hist(train_df.logerror, bins = 50)
plt.xlabel('logerror', fontsize = 16)

display logerror histogram without outlier

In [ ]:
upperlimit = np.percentile(train_df.logerror, 99)
lowerlimit = np.percentile(train_df.logerror, 1)

plt.figure(figsize = (16, 6))
plt.hist(train_df.query('logerror < {} and logerror > {}'.format(upperlimit, lowerlimit)).logerror, bins = 50)
plt.xlabel('logerror', fontsize = 16)

## Key feature - taxvaluedollarcnt

In [ ]:
# when there is NaN in your column, use query to remove NaN for data display 
plt.figure(figsize = (16, 6))
plt.hist(train_df.query('taxvaluedollarcnt == taxvaluedollarcnt').taxvaluedollarcnt, bins = 50)
#plt.hist(train_df.taxvaluedollarcnt, bins = 50)
plt.xlabel('taxvaluedollarcnt', fontsize = 16)

In [ ]:
upperlimit = np.percentile(train_df.query('taxvaluedollarcnt == taxvaluedollarcnt').taxvaluedollarcnt.values, 99)
lowerlimit = np.percentile(train_df.query('taxvaluedollarcnt == taxvaluedollarcnt').taxvaluedollarcnt.values, 1)

plt.figure(figsize = (16, 6))
plt.hist(train_df.query('taxvaluedollarcnt < {} and taxvaluedollarcnt > {}'.format(upperlimit, lowerlimit)).taxvaluedollarcnt, bins = 50)
plt.xlabel('taxvaluedollarcnt', fontsize = 16)

## Key Feature - Lot size

In [ ]:
plt.figure(figsize = (16, 6))
plt.hist(train_df.query('lotsizesquarefeet == lotsizesquarefeet').lotsizesquarefeet, bins = 50)
plt.xlabel('lotsizesquarefeet', fontsize = 16)

In [ ]:
plt.figure(figsize = (16, 6))
plt.hist(train_df.query('lotsizesquarefeet > 0 and lotsizesquarefeet < 30000').lotsizesquarefeet, bins = 50)
plt.xlabel('lotsizesquarefeet', fontsize = 16)

## Key feature - built year

In [ ]:
plt.figure(figsize = (16, 6))
plt.hist(train_df.query('yearbuilt == yearbuilt').yearbuilt, bins = 50)
plt.xlabel('yearbuilt', fontsize = 16)
plt.show()

## Multivariate EDA - logerror vs. tax value

In [ ]:
plt.figure(figsize = (16, 16))
sns.jointplot('taxvaluedollarcnt', 'logerror', train_df.query('taxvaluedollarcnt==taxvaluedollarcnt'), 
              size = 10, kind='reg', )
plt.xlabel('taxvaluedollarcnt', fontsize = 16)
plt.ylabel('logerror', fontsize = 16)
plt.show()

There is no clear relation between logerror and taxvaluedollarcnt

## add abs_logerror as the new feature

In [ ]:
train_df['abs_logerror'] = train_df.logerror.abs()

In [ ]:
plt.figure(figsize = (16, 16))
sns.jointplot('taxvaluedollarcnt', 'abs_logerror', train_df.query('taxvaluedollarcnt == taxvaluedollarcnt'), 
              size = 10, kind = 'reg')
plt.xlabel('taxvaluedollarcnt', fontsize = 16)
plt.ylabel('abs_logerror', fontsize = 16)
plt.show()

for high value house, taxvaluedollarcnt could help a bit

matplotlib scatter plot with alpha 

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(train_df.taxvaluedollarcnt, train_df.abs_logerror, alpha=0.1)
plt.xlabel('taxvaluedollarcnt', fontsize = 16)
plt.ylabel('abs_logerror', fontsize = 16)
plt.show()

zoom into detail

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(train_df.taxvaluedollarcnt, train_df.abs_logerror, alpha=0.1)
plt.xlabel('taxvaluedollarcnt', fontsize = 16)
plt.ylabel('abs_logerror', fontsize = 16)
plt.xlim(0, 1000000)
plt.ylim(0, 1)
plt.show()

## Multivariate EDA - logerror vs. built year

### simple cross-plot between abs_logerror and built year

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(train_df.yearbuilt, train_df.abs_logerror, alpha=0.1)
plt.xlabel('year', fontsize = 16)
plt.ylabel('abs_logerror', fontsize = 16)
plt.show()

### similar cross-plot with seaborn

In [ ]:
# plt.figure(figsize = (16, 6))
sns.jointplot('yearbuilt', 'abs_logerror', train_df.query('yearbuilt==yearbuilt'), 
              size = 9, kind='reg')
plt.xlabel('year', fontsize = 16)
plt.ylabel('abs_logerror', fontsize = 16)
plt.show()

Aggregate abs_logerror regarding built year

In [ ]:
train_df_logerrorAggYear = train_df.groupby('yearbuilt').abs_logerror.mean().reset_index()

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(train_df_logerrorAggYear.yearbuilt, train_df_logerrorAggYear.abs_logerror)
plt.xlabel('year', fontsize = 16)
plt.ylabel('abs_logerror', fontsize = 16)
plt.show()

In [ ]:
sns.jointplot('yearbuilt', 'abs_logerror', 
             train_df_logerrorAggYear, size = 10, kind = 'reg')
plt.xlabel('year', fontsize = 16)
plt.ylabel('abs_logerror', fontsize = 16)
plt.show()

In [ ]:
plt.figure(figsize = (16, 10))

plt.scatter(train_df_logerrorAggYear.yearbuilt, train_df_logerrorAggYear.abs_logerror, 
            label='data', color='red', marker='o', alpha=.5)
# sns.regplot(train_df_logerrorAggYear.yearbuilt, train_df_logerrorAggYear.abs_logerror, 
#             scatter=None, color='blue', label='order 1')
sns.regplot(train_df_logerrorAggYear.yearbuilt, train_df_logerrorAggYear.abs_logerror, 
            scatter=None, order=2, color='green', label='order 2')
sns.regplot(train_df_logerrorAggYear.yearbuilt, train_df_logerrorAggYear.abs_logerror, 
            scatter=None, order=3, color='purple', label='order 3')

plt.xlabel('year', fontsize = 16)
plt.ylabel('abs_logerror', fontsize = 16)

plt.legend(loc='upper right')
plt.show()

new houses are easier to predict

parse year and month from transactiondate

In [ ]:
train_df_copy = train_df.copy()
train_df_copy['transactiondate'] = pd.to_datetime(train_df_copy['transactiondate'])

In [ ]:
train_df_copy['transactionyear'] = train_df_copy['transactiondate'].dt.year
train_df_copy['transactionmonth'] = train_df_copy['transactiondate'].dt.month

In [ ]:
train_df_logerrorTransactionyear = train_df_copy.groupby('transactionyear').abs_logerror.mean().reset_index()

In [ ]:
train_df_logerrorTransactionyear

In [ ]:
train_df_logerrorTransactionmonth = train_df_copy.groupby('transactionmonth').abs_logerror.mean().reset_index()

In [ ]:
train_df_logerrorTransactionmonth

In [ ]:
plt.figure(figsize = (16, 8))

plt.plot(train_df_logerrorTransactionmonth.transactionmonth, train_df_logerrorTransactionmonth.abs_logerror, 
         color='red', marker='o')
plt.show()

## Multivariate EDA - lot size vs. tax value

In [ ]:
sns.jointplot('calculatedfinishedsquarefeet','taxamount',
              train_df,size=10,kind='reg')

In [ ]:
sns.jointplot('calculatedfinishedsquarefeet','taxamount',
              train_df,size=10,kind='kde')
# plt.xlim(0, 10000)
# plt.ylim(0, 10000)

## Multivariate EDA - tax value vs. tax amount

In [ ]:
sns.jointplot('taxvaluedollarcnt','taxamount',
              train_df,size=10,kind='reg')

## Multivariate EDA - logerror vs. architecturalstyletypeid

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot(data=train_df,
            x='architecturalstyletypeid',
            y='abs_logerror')

In [ ]:
plt.figure(figsize=(20, 10))
sns.violinplot(data=train_df,
            x='architecturalstyletypeid',
            y='abs_logerror')

Zillow model is not ideal for architectural style 7, more time could be spent on that

In [ ]:
plt.figure(figsize=(20, 10))
sns.boxplot(data=train_df,
            x='architecturalstyletypeid',
            y='taxvaluedollarcnt')

In [ ]:
plt.figure(figsize=(20, 10))
sns.violinplot(data=train_df,
            x='architecturalstyletypeid',
            y='taxvaluedollarcnt')

# Find Missing Values

In [ ]:
train_df.shape

In [ ]:
missing_value_df = train_df.isnull().sum(axis = 0).reset_index()
missing_value_df.columns = ['column_name', 'missing_count']
missing_value_df = missing_value_df.loc[missing_value_df['missing_count'] > 0]
missing_value_df = missing_value_df.sort_values(by = 'missing_count')

index = np.arange(missing_value_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,20))
rects = ax.barh(index, missing_value_df.missing_count.values / train_df.shape[0], color='blue')
ax.set_yticks(index)
ax.set_yticklabels(missing_value_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Fraction of missing values")
ax.set_title("Fractions of missing values in each column")
plt.show()

# Data Preprocessing

## Group Variables into different groups

In [ ]:
cat_vars = list(train_df.dtypes[train_df.dtypes=='object'].index)
num_vars = list(train_df.dtypes[train_df.dtypes=='int64'].index) + list(train_df.dtypes[train_df.dtypes=='float64'].index)

id_var = 'id'
target_var = 'logerror'
num_vars.remove('parcelid')
num_vars.remove('logerror')
num_vars.remove('abs_logerror')
cat_vars.remove('transactiondate')

dt_vars=['transactiondate']

print("Categorical features:", cat_vars)
print( "Numerical features:", num_vars)
print( "Datetime features:", dt_vars)
print( "ID: {}, target: {}" .format( id_var, target_var))

## Create New Numerical Features

In [ ]:
properties['finished_sq_ratio'] = properties[['calculatedfinishedsquarefeet','lotsizesquarefeet']].apply(
    lambda x : x[0]/x[1] if x[1] > 0 else -999999, axis = 1)

properties['taxvalue_per_sq'] = properties[['taxvaluedollarcnt','calculatedfinishedsquarefeet']].apply(
    lambda x : x[0]/x[1] if x[1] > 0 else -999999, axis = 1)

properties['structure_tax_ratio'] = properties[['structuretaxvaluedollarcnt','taxvaluedollarcnt']].apply(
    lambda x : x[0]/x[1] if x[1] > 0 else -999999, axis = 1)

properties['landtax_per_sq'] = properties[['landtaxvaluedollarcnt','lotsizesquarefeet']].apply(
    lambda x : x[0]/x[1] if x[1] > 0 else -999999, axis = 1)

properties['assessmentyear_to_builtyear']= properties[['assessmentyear','yearbuilt']].apply(
    lambda x : x[0]-x[1] if x[0] > 0 and x[1] > 0 else -999999, axis = 1)


num_to_num_vars = ['finished_sq_ratio','taxvalue_per_sq','structure_tax_ratio',
                   'landtax_per_sq','assessmentyear_to_builtyear']

### update train_df with new features

In [ ]:
train_df = pd.merge(train_df, properties[num_to_num_vars + ['parcelid']], how='left', on='parcelid')

# Categorical features: Label Encoding


## Label Encoding only works for tree models, will need One Hot Encoding

In [ ]:
LE = preprocessing.LabelEncoder()

LE_vars=[]
LE_map=dict()
for cat_var in cat_vars:
    print ("Label Encoding {}".format(cat_var))
    LE_var = cat_var + '_le'
    properties[LE_var] = LE.fit_transform(properties[cat_var].astype(str).fillna('none'))
    LE_vars.append(LE_var)
    LE_map[cat_var]=LE.classes_
    
print ("Label-encoded feaures: {}".format(LE_vars))

# Categorical features: One Hot Encoding

In [ ]:
OHE = preprocessing.OneHotEncoder(sparse=True)
start=time.time()
OHE.fit(properties[LE_vars])
OHE_sparse=OHE.transform(properties[LE_vars])
                                   
print ('One-hot-encoding finished in {} seconds'.format(time.time()-start))

OHE_vars = [var[:-3] + '_' + str(level).replace(' ','_')\
                for var in cat_vars for level in LE_map[var] ]

print ("OHE_sparse size :" ,OHE_sparse.shape)
print ("One-hot encoded catgorical feature samples : {}".format(OHE_vars[:100]))

# Modeling

In [ ]:
# start with minimum features
train_df = train.merge(properties, how='left', on='parcelid')

In [ ]:
full_vars = num_vars + LE_vars 
train_x = train_df[full_vars]
train_y = train_df['logerror'].values.astype(np.float32)

test_x = properties[full_vars]

# xgboost params
xgb_params = {
    'eta': 0.05,
    'max_depth': 4,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'silent': 1,
    'seed': 1234
}

dtrain = xgb.DMatrix(train_x, train_y)
dtest = xgb.DMatrix(test_x)

# cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   nfold=5,
                   num_boost_round=10000,
                   early_stopping_rounds=50,
                   verbose_eval=10, 
                   show_stdv=False,
                   seed = 1234
                  )

## best score and best round
best_iteration = len(cv_result)
best_score = cv_result['test-mae-mean'].min()
print("Best score {}, best iteration {}".format(best_score,best_iteration))


In [ ]:
model = xgb.train(dict(xgb_params, silent = 1), dtrain, num_boost_round = best_iteration)
pred = model.predict(dtest)
y_pred=[]

for i,predict in enumerate(pred):
    y_pred.append(str(round(predict,4)))
y_pred=np.array(y_pred)

output = pd.DataFrame({'ParcelId': properties['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})
# set col 'ParceID' to first col
cols = output.columns.tolist()
cols = cols[-1:] + cols[:-1]
output = output[cols]
from datetime import datetime
output.to_csv('outputs/sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index = False)

print ("Finished")

In [ ]:
feature_imporantce = pd.Series(model.get_fscore()).sort_values(ascending = True)
feature_imporantce.plot.barh(x='feature_name',figsize=(8,15))

In [ ]:
full_vars = num_vars + LE_vars + num_to_num_vars
train_x = train_df[full_vars]
train_y = train_df['logerror'].values.astype(np.float32)

test_x = properties[full_vars]

# xgboost params
xgb_params = {
    'eta': 0.05,
    'max_depth': 4,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'silent': 1,
    'seed': 1234
}

dtrain = xgb.DMatrix(train_x, train_y)
dtest = xgb.DMatrix(test_x)

# cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   nfold=5,
                   num_boost_round=10000,
                   early_stopping_rounds=50,
                   verbose_eval=10, 
                   show_stdv=False,
                   seed = 1234
                  )

## best score and best round
best_iteration = len(cv_result)
best_score = cv_result['test-mae-mean'].min()
print("Best score {}, best iteration {}".format(best_score,best_iteration))

In [ ]:
model = xgb.train(dict(xgb_params, silent = 1), dtrain, num_boost_round = best_iteration)
pred = model.predict(dtest)
y_pred=[]

for i,predict in enumerate(pred):
    y_pred.append(str(round(predict,4)))
y_pred=np.array(y_pred)

output = pd.DataFrame({'ParcelId': properties['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})
# set col 'ParceID' to first col
cols = output.columns.tolist()
cols = cols[-1:] + cols[:-1]
output = output[cols]
from datetime import datetime
output.to_csv('outputs/sub_with_engineered_features{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)

print ("Finished")

In [ ]:
feature_imporantce = pd.Series(model.get_fscore()).sort_values(ascending = True)
feature_imporantce.plot.barh(x='feature_name',figsize=(8,15))

# XGBoost  tuning
## Manuanl Tuning

In [ ]:
full_vars = num_vars + LE_vars
train_df = pd.merge(train, properties,
                     how='left', on='parcelid')

train_x = train_df[full_vars]
train_y = train_df['logerror'].values.astype(np.float32)

xgtrain = xgb.DMatrix(train_x, train_y)
xgtest = xgb.DMatrix(properties[full_vars])

### max_depth

In [ ]:
%%time
xgb_scores = pd.DataFrame()
scores = []

for max_depth in [3,4,5,6,7,8,9,10]:

    params = dict()
    params['objective'] = 'reg:linear'
    params['eta'] = 0.1
    params['max_depth'] = max_depth
    params['min_child_weight'] = 1
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgtrain,
                        num_boost_round=1000000,
                        nfold=5,
                        metrics={'mae'},
                        seed=1234,
                        callbacks=[xgb.callback.early_stop(50)],
                        verbose_eval=50)
    best_iteration = len(cv_results)
    best_score = cv_results['test-mae-mean'].min()
    print (max_depth,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_max_depth = int(pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['max_depth'].values[0])
print ('best max_depth is', best_max_depth)

### min_child_weight

In [ ]:
%%time
xgb_scores = pd.DataFrame()
scores = []

for min_child_weight in [1,3,10,30,50,75,100]:

    params = dict()
    params['objective'] = 'reg:linear'
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = min_child_weight
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgtrain,
                        num_boost_round=1000000,
                        nfold=5,
                        metrics={'mae'},
                        seed=1234,
                        callbacks=[xgb.callback.early_stop(50)],
                        verbose_eval=50)
    best_iteration = len(cv_results)
    best_score = cv_results['test-mae-mean'].min()
    print (min_child_weight,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_min_child_weight = int(pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).sort_values(by='score',ascending=True)['min_child_weight'].values[0])
print ('best min_child_weight is', best_min_child_weight)

### colsample_bytree

In [ ]:
%%time
xgb_scores = pd.DataFrame()
scores = []

for colsample_bytree in [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:

    params = dict()
    params['objective'] = 'reg:linear'
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['colsample_bytree'] = colsample_bytree
    params['subsample'] = 1
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgtrain,
                        num_boost_round=1000000,
                        nfold=5,
                        metrics={'mae'},
                        seed=1234,
                        callbacks=[xgb.callback.early_stop(50)],
                        verbose_eval=50)
    best_iteration = len(cv_results)
    best_score = cv_results['test-mae-mean'].min()
    print (colsample_bytree,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_colsample_bytree = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).\
                sort_values(by='score',ascending=True)['colsample_bytree'].values[0]
print ('best colsample_bytree is', best_colsample_bytree)

### subsample

In [ ]:
%%time
xgb_scores = pd.DataFrame()
scores = []

for subsample in [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:

    params = dict()
    params['objective'] = 'reg:linear'
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['colsample_bytree'] = best_colsample_bytree
    params['subsample'] = subsample
    params['gamma'] = 0
    params['seed']=1234

    cv_results = xgb.cv(params, xgtrain,
                        num_boost_round=1000000,
                        nfold=5,
                        metrics={'mae'},
                        seed=1234,
                        callbacks=[xgb.callback.early_stop(50)],
                        verbose_eval=50)
    best_iteration = len(cv_results)
    best_score = cv_results['test-mae-mean'].min()
    print (subsample,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_subsample = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).\
                sort_values(by='score',ascending=True)['subsample'].values[0]
print ('best subsample is', best_subsample)

### gamma

In [ ]:
%%time
xgb_scores = pd.DataFrame()
scores = []

for gamma in [0,0.1,0.2,0.5,1,1.25,1.5,1.75,2]:

    params = dict()
    params['objective'] = 'reg:linear'
    params['eta'] = 0.1
    params['max_depth'] = best_max_depth
    params['min_child_weight'] = best_min_child_weight
    params['colsample_bytree'] = best_colsample_bytree
    params['subsample'] = best_subsample
    params['gamma'] = gamma
    params['seed']=1234

    cv_results = xgb.cv(params, xgtrain,
                        num_boost_round=1000000,
                        nfold=5,
                        metrics={'mae'},
                        seed=1234,
                        callbacks=[xgb.callback.early_stop(50)],
                        verbose_eval=50)
    best_iteration = len(cv_results)
    best_score = cv_results['test-mae-mean'].min()
    print (gamma,best_score,best_iteration)
    scores.append([best_score,params['eta'],params['max_depth'],params['min_child_weight'],
                      params['colsample_bytree'],params['subsample'],params['gamma'],best_iteration])
xgb_scores = pd.concat([xgb_scores, pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration'])])    
best_gamma = pd.DataFrame(scores,columns=['score','eta','max_depth','min_child_weight',
                                   'colsample_bytree','subsample','gamma','best_iteration']).\
                    sort_values(by='score',ascending=True)['gamma'].values[0]
print ('best gamma is', best_subsample)

### XGBoost model training with manually tuned parameters

In [ ]:
params = dict()
params['objective'] = 'reg:linear'
params['eta'] = 0.1
params['max_depth'] = best_max_depth
params['min_child_weight'] = best_min_child_weight
params['colsample_bytree'] = best_colsample_bytree
params['subsample'] = best_subsample
params['gamma'] = best_gamma
params['seed']=1234



model = xgb.train(params, xgtrain, num_boost_round=best_iteration)
pred = model.predict(xgtest)
y_pred=[]

for i,predict in enumerate(pred):
    y_pred.append(str(round(predict,4)))
y_pred=np.array(y_pred)

output = pd.DataFrame({'ParcelId': properties['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})
# set col 'ParceID' to first col
cols = output.columns.tolist()
cols = cols[-1:] + cols[:-1]
output = output[cols]
from datetime import datetime
output.to_csv('output/sub_manually_tuned{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)

print ("Finished")